In [36]:
import json

import click

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

from utils import chat_llm

In [37]:
html2json_prompt_path = "../../prompt/ours/html2json.txt"

with open(html2json_prompt_path, "r") as f:
    html2json_prompt = f.read()
    
print(html2json_prompt)

In [38]:
chromedriver_path = "/home/lornd/Downloads/chromedriver"

def get_webdriver():
    service = Service(chromedriver_path)
    options = webdriver.ChromeOptions()
    options.add_argument("disable-gpu")
    options.add_argument("no-sandbox")

    chromedriver = webdriver.Chrome(service=service, options=options)
    chromedriver.implicitly_wait(5)
    chromedriver.maximize_window()
    chromedriver.implicitly_wait(5)

    return chromedriver

driver = get_webdriver()


In [39]:
def is_hidden(element):
    if element is None:
        return True
    style = element.get("style", "")
    if "display: none" in style or "visibility: hidden" in style:
        return True
    css_class = element.get("class", "")
    # click.echo(click.style(f"Style: {style}", fg="green"))
    # click.echo(click.style(f"class: {css_class}", fg="blue"))
    if "hide" in css_class:
        return True
    return False


def delete_hidden_elements(html):
    soup = BeautifulSoup(html, "html.parser")
    all_elements = soup.find_all(True)
    
    elements_to_remove = []
    for element in all_elements:
        if is_hidden(element):
            elements_to_remove.append(element)

    for element in elements_to_remove:
        element.decompose()
    
    new_html = str(soup)
    return new_html

In [40]:
def html2json(env_name, driver):
    miniwob_dir = "/home/lornd/Projects/WebNavigation/env/miniwob/miniwob_interface/html/miniwob"

    base_url = f"file://{miniwob_dir}"
    
    url = f"{base_url}/{env_name}.html"
    
    driver.get(url)
    
    driver.execute_script("core.startEpisodeReal();")
    
    task = driver.execute_script("return core.getUtterance();") # 获取当前网页需要做的事情 
    
    playground = driver.find_element(By.ID, "area")
        
    html = playground.get_attribute("outerHTML")
    
    html = delete_hidden_elements(html)
    
    click.echo(click.style(html, fg="green"))
    
    messages = [
        {"role": "system", "content": html2json_prompt},
        {"role": "user", "content": html}
    ]
    
    model = "gpt-3.5-turbo"
    response = chat_llm(model, messages)
    content = response.content
    role = response.role
    return task, content

In [43]:
ignored_tags = ["br"]

def dfs(element):
    if element.tag_name in ignored_tags:
        return None
    has_children = driver.execute_script("return arguments[0].children.length > 0;", element)
    if has_children:
        all_child_elements = element.find_elements(By.XPATH, "./*")
        children = []

        for child in all_child_elements:
            child_result = dfs(child)
            if child_result is not None:
                children.append(child_result)

        return {
            "tag": element.tag_name,
            "children": children
        }
    else:
        return {
            "tag": element.tag_name,
            "content": element.text
        }


def html2json_dfs(env_name, driver):
    miniwob_dir = "/home/lornd/Projects/WebNavigation/env/miniwob/miniwob_interface/html/miniwob"

    base_url = f"file://{miniwob_dir}"

    url = f"{base_url}/{env_name}.html"

    driver.get(url)

    driver.execute_script("core.startEpisodeReal();")

    task = driver.execute_script("return core.getUtterance();")  # 获取当前网页需要做的事情

    playground = driver.find_element(By.ID, "area")

    return dfs(playground)

In [44]:
# goal, playground = html2json("choose-list", driver)
# click.echo(click.style(f"goal: {goal}", fg="blue"))
# click.echo(click.style(f"playground: {playground}", fg="green"))
res = html2json_dfs("choose-list", driver)
print(json.dumps(res, indent=4))

In [45]:
# print(html2json("click-button", driver))
# print(html2json_dfs("click-button", driver))
res = html2json_dfs("click-button", driver)
print(json.dumps(res, indent=4))

In [ ]:
print(html2json("enter-password", driver))

In [15]:
print(html2json("social-media", driver))